# Tensorflow Core Learning Algorithms 2

In [287]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

## Classification

Now that we've covered linear regression, it is time to talk about classification. Where regression was used to predict a numeric value, classification is used to seperate data points into classes of different labels. In this example we will use a TensorFlow estimator to classify flowers.

Since we've touched on how estimators work earlier I'll go a bit quicker through this example.

This section is based on the following guide from the TensorFlow website.
https://www.tensorflow.org.tutorials/estimator/premade

### Imports and Setup

In [271]:
print(tf.__version__)

2.15.0


### Dataset
This specific dataset seperates floweres into 3 different classes of species.

- Setosa
- Versicolor
- Virginica

The information about each flower is the following.

- sepal length
- sepal width
- petal length
- petal width

In [272]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Let's define some constants to help us later on

In [273]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe



Let's take a look at our data

In [274]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Now, we can pop the species column off that use as our label

In [275]:
train_y = train.pop('Species')
test_y = test.pop('Species')

train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [276]:
train.shape

(120, 4)

In [277]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

### Input Function
Remember that nasty input function we created earlier. Well we need to make another one here! Fortunately for us this one is a little easier to digest.

In [278]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()

    return dataset.batch(batch_size)

### Feature Columns

In [279]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


### Building the Model
And now we are ready to choose a model. For classification tasks, there are variety of different estimators/models that we can pick from.

Some options are listed below.

- `DNNClassifier` (Deep Neural Network)
- `LinearClassifier`

We can choose either model but DNN seems to be the best choice. This is because we may not be able to find a linear correspondence in our data.

So let's build a model!

In [280]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[30,10],
    # The model must choose between 3 classes.
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


What we've just done is created a deep neural network that has two hidden layers. These layers have 30 and 10 neurons respectively. This is the number of neurons of the TensorFlow official tutorial uses so we'll stick with it. However, it is worth mentioning that the number of hidden neurons is an arbitrary number and many experiments and tests are usually done to determine the best choice for these values. Try playing around with the number of hidden neurons and see if your results change.

`tf.estimator` stores a lot of pre-made models from TensorFlow.

More explanation for neural network stuff later on.

### Training
Now, it's time to train the model!

In [281]:
classifier.train(input_fn=lambda: input_fn(train, train_y, training=True),
                 steps=5000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.6863942, step = 0
INFO:tensorflow:global_step/sec: 1213.3
INFO:tensorflow:loss = 1.14976, step = 100 (0.082 sec)
INFO:tensorflow:global_step/sec: 2117.3
INFO:tensorflow:loss = 1.0080228, step = 200 (0.047 sec)
INFO:tensorflow:global_step/sec: 2085.76
INFO:tensorflow:loss = 0.9451312, step = 300 (0.048 sec)
INFO:tensorflow:global_step/sec: 2092.66
INFO:tensorflow:loss = 0.9066018, step = 400 (0.048 sec)
INFO:tensorflow:global_step/sec: 2056.09
INFO:tensorflow:loss = 

The lower the "loss" the better.

### Evaluation
Let's see how it performs

In [282]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2025-08-26T09:47:13
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.10611s
INFO:tensorflow:Finished evaluation at 2025-08-26-09:47:13
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.93333334, average_loss = 0.44920492, global_step = 5000, loss = 0.44920492
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe/model.ckpt-5000


In [283]:
print("\nTest set accuracy: {accuracy:0.3f}\n".format(**eval_result))


Test set accuracy: 0.933



### Predictions
Let's see how this model predicts

In [284]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric value as prompted.")
for feature in features:
    valid = True
    while valid:
        val = input(feature + ": ")
        print('"{}": "{}"'.format(feature, val))
        if not val.isdigit(): valid = False

    predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Predictions is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric value as prompted.
"SepalLength": "2.1"
"SepalWidth": "3.2"
"PetalLength": "4.5"
"PetalWidth": "7.2"
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([0.3716182 , 0.09661669, 4.073316  ], dtype=float32), 'probabilities': array([0.0236543 , 0.01796712, 0.95837855], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Predictions is "Virginica" (95.8%)


In [285]:
# Some example input and and expected classes you can try above
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

In [286]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict_x))
i = 0
for pred_dict in predictions:
    i += 1
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Predictions is "{}" ({:.1f}%), expected is {}'.format(
        SPECIES[class_id], 100 * probability, expected[i-1]))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/3q/7940jm5955s9rqks86c04zcc0000gn/T/tmp86vak7qe/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Predictions is "Setosa" (84.5%), expected is Setosa
Predictions is "Versicolor" (52.9%), expected is Versicolor
Predictions is "Virginica" (60.6%), expected is Virginica
